# Camshift 

In [1]:
import numpy as np 
import cv2
  
# Read the input video 
cap = cv2.VideoCapture('Input.mp4') 
frame = None
frame_width = int(cap.get(3)) 
frame_height = int(cap.get(4)) 
size = (frame_width, frame_height)
ret, frame = cap.read()
img = cv2.imread('object.png')
# Getting the coordinates
x = 70
y = 200
width = 260 - x
height = 510 - y
track_window = (x, y, width, height) 
result = cv2.VideoWriter('Output.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)
if frame is not None:
    # To draw rectangle
    cv2.rectangle(frame, (y + height, y + height), (y, y), (0, 255, 0), 0)
    # Getting the region of Interest
    roi = frame[y:y + height, x:x + width] 
    # convert ROI from BGR to HSV format 
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV) 
    # Obtaining mask 
    mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255))) 
    # Getting the histogram for ROI
    roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
    cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX) 
    # Termination criteria,  
    # either 15 iteration or move by atleast 2 pt 
    term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 1) 
    while(1):   
        ret, frame = cap.read()  
    # Displaying the original video
        cv2.imshow('Original', frame) 
        ret1, frame1 = cv2.threshold(frame, 180, 155, cv2.THRESH_TOZERO_INV) 
        hsv = cv2.cvtColor(frame1,cv2.COLOR_BGR2HSV) 
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1) 
    # Applying Camshift
        ret2, track_window = cv2.CamShift(dst, track_window, term_crit) 
  
    # Inseting the rectange on the image
        pts = cv2.boxPoints(ret2)  
        pts = np.int0(pts)  
        Result = cv2.polylines(frame, [pts], True,(255, 255, 0), 2) 
    # Displaying the Final video
    
        cv2.imshow('Final', Result) 
        result.write(frame)
    # Press ESC to stop 
        k = cv2.waitKey(20) & 0xff
        if k == 27: 
            break
else:
    print("Not able to open video")
# Releasing the cap object 
cap.release() 
result.release() 
# closing all opened windows 
cv2.destroyAllWindows() 